In [1]:
from datetime import datetime
import xarray as xr

from mhwpype import ShiftedBaseline
from mhwpype.utils import update_names, reformat_longitude, assign_depth
from mhwpype.core import build_heat_spike_mask
from mhwpype.core import build_mhw_mask, group_from_mask, group_stats

In [2]:
filepath = 'data/sst.day.mean.1981-2024.nc'
ds = xr.open_dataset(filepath)

ds = update_names(ds)
ds['longitude'] = reformat_longitude(ds.longitude)
ds = ds.sortby(['time','latitude','longitude'])
ds = ds.sel(latitude= slice(41.5,46.5), longitude=slice(-130,-122))
ds = assign_depth(ds,0)

In [3]:
doi = ds.sea_water_temperature.sel(time = slice(datetime(2015,1,1), datetime(2023,12,31,23,59,59)))

In [4]:
Shifted = ShiftedBaseline()
shifted_climatology = Shifted.build_daily_climatology(ds.sea_water_temperature, range(2015,2025), 30, 5, True)
shifted_threshold = Shifted.build_daily_threshold(ds.sea_water_temperature, [0.9], 'linear', range(2015,2025), 30, 5, True)

In [5]:
heat_spike_mask = build_heat_spike_mask(doi, shifted_threshold, core_dim = 'time')

In [6]:
mhw_mask = build_mhw_mask(doi,shifted_threshold, core_dim = 'time')

In [7]:
mhw_groups = group_from_mask(mhw_mask)

In [8]:
mhw_stats = group_stats(doi, shifted_climatology, shifted_threshold, mhw_groups, core_dim='time')

In [9]:
mhw_stats

<xarray.Dataset> Size: 46MB
Dimensions:                                (event_id: 1153, depth: 1,
                                            latitude: 20, longitude: 25,
                                            quantile: 1)
Coordinates:
  * depth                                  (depth) int64 8B 0
  * quantile                               (quantile) float64 8B 0.9
  * event_id                               (event_id) datetime64[ns] 9kB 2015...
  * latitude                               (latitude) float32 80B 41.62 ... 4...
  * longitude                              (longitude) float32 100B -129.9 .....
Data variables:
    event_start                            (event_id, depth, latitude, longitude, quantile) datetime64[ns] 5MB ...
    event_end                              (event_id, depth, latitude, longitude, quantile) datetime64[ns] 5MB ...
    event_duration                         (event_id, depth, latitude, longitude, quantile) float64 5MB ...
    event_peak_sea_water_temperature       (event_id, depth, latitude, longitude, quantile) float64 5MB ...
    event_peak_sea_water_temperature_time  (event_id, depth, latitude, longitude, quantile) datetime64[ns] 5MB ...
    event_peak_anomaly                     (event_id, depth, latitude, longitude, quantile) float64 5MB ...
    event_peak_anomaly_time                (event_id, depth, latitude, longitude, quantile) datetime64[ns] 5MB ...
    event_intensity_max                    (event_id, depth, latitude, longitude) float32 2MB ...
    event_intensity_mean                   (event_id, depth, latitude, longitude) float32 2MB ...
    event_intensity_stdev                  (event_id, depth, latitude, longitude) float32 2MB ...
    event_intensity_cumulative             (event_id, depth, latitude, longitude) float32 2MB ...
    event_peak_category                    (event_id, depth, latitude, longitude, quantile) float64 5MB ...